In [1]:

using MLBase
using Plots
using DecisionTree
using Distances
using LIBSVM
using CSV
using DataFrames
using Random

In [2]:
data = CSV.read("Iris.csv", DataFrame)

Row,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
,Int64,Float64,Float64,Float64,Float64,String15
1,1,5.1,3.5,1.4,0.2,Iris-setosa
2,2,4.9,3.0,1.4,0.2,Iris-setosa
3,3,4.7,3.2,1.3,0.2,Iris-setosa
4,4,4.6,3.1,1.5,0.2,Iris-setosa
5,5,5.0,3.6,1.4,0.2,Iris-setosa
6,6,5.4,3.9,1.7,0.4,Iris-setosa
7,7,4.6,3.4,1.4,0.3,Iris-setosa
8,8,5.0,3.4,1.5,0.2,Iris-setosa
9,9,4.4,2.9,1.4,0.2,Iris-setosa


In [3]:

X = Matrix(data[:,2:5])
irislabels = data[:,6]

150-element PooledArrays.PooledVector{String15, UInt32, Vector{UInt32}}:
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 ⋮
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"

In [4]:
#normalizing the data labels
irislabelsmap = labelmap(irislabels)
y = labelencode(irislabelsmap,irislabels)

150-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

In [5]:
function perclass_splits(y,at)
    uids = unique(y)
    keepids = []
    for ui in uids
        curids = findall(y.==ui)
        rowids = randsubseq(curids, at)
        push!(keepids,rowids...)
    end
    return keepids
end

perclass_splits (generic function with 1 method)

In [6]:
findaccuracy(predictedvals, groundtruthvals) = sum(predictedvals.==groundtruthvals)/length(groundtruthvals)

findaccuracy (generic function with 1 method)

In [7]:
trainids = perclass_splits(y,0.7)
testids = setdiff(1:length(y),trainids)

45-element Vector{Any}:
   2
   3
  11
  13
  16
  20
  22
  33
  34
  35
   ⋮
 130
 132
 140
 142
 145
 146
 147
 149
 150

In [8]:
assing_class(predictedvalue) = argmin(abs.(predictedvalue .- [1,2,3]))

assing_class (generic function with 1 method)

In [9]:
Xtrain = X[trainids,:]
ytrain = y[trainids]

105-element Vector{Int64}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 3
 3
 3
 3
 3
 3
 3
 3

Decision tree!!!! using decisiontree package

In [10]:
model_DT = DecisionTreeClassifier(max_depth=2)
DecisionTree.fit!(model_DT, Xtrain, ytrain)

DecisionTreeClassifier
max_depth:                2
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3]
root:                     Decision Tree
Leaves: 3
Depth:  2

In [11]:
q = X[testids,:];
pred_DT = DecisionTree.predict(model_DT,q)
findaccuracy(pred_DT,y[testids])

0.9555555555555556

Random forest!!!!

In [12]:
model_RF = RandomForestClassifier(n_trees = 20)
DecisionTree.fit!(model_RF,Xtrain, ytrain)

RandomForestClassifier
n_trees:             20
n_subfeatures:       -1
partial_sampling:    0.7
max_depth:           -1
min_samples_leaf:    1
min_samples_split:   2
min_purity_increase: 0.0
classes:             [1, 2, 3]
ensemble:            Ensemble of Decision Trees
Trees:      20
Avg Leaves: 6.75
Avg Depth:  4.55

In [13]:
pred_RF = DecisionTree.predict(model_RF, q)
findaccuracy(pred_RF, y[testids])

0.9777777777777777

SVM

In [14]:
model_SVM = svmtrain(Xtrain', ytrain)

LIBSVM.SVM{Int64, LIBSVM.Kernel.KERNEL}(SVC, LIBSVM.Kernel.RadialBasis, nothing, 4, 105, 3, [1, 2, 3], Int32[1, 2, 3], Float64[], Int32[], LIBSVM.SupportVectors{Vector{Int64}, Matrix{Float64}}(39, Int32[6, 17, 16], [1, 1, 1, 1, 1, 1, 2, 2, 2, 2  …  3, 3, 3, 3, 3, 3, 3, 3, 3, 3], [4.4 4.3 … 5.8 6.5; 2.9 3.0 … 2.7 3.0; 1.4 1.1 … 5.1 5.2; 0.2 0.1 … 1.9 2.0], Int32[7, 10, 11, 14, 17, 18, 35, 38, 39, 40  …  87, 88, 90, 92, 93, 96, 97, 101, 103, 105], LIBSVM.SVMNode[LIBSVM.SVMNode(1, 4.4), LIBSVM.SVMNode(1, 4.3), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 5.1), LIBSVM.SVMNode(1, 4.8), LIBSVM.SVMNode(1, 7.0), LIBSVM.SVMNode(1, 5.7), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 4.9)  …  LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 5.6), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.2), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.3), LIBSVM.SVMNode(1, 6.1), LIBSVM.SVMNode(1, 6.0), LIBSVM.SVMNode(1, 5.8), LIBSVM.SVMNode(1, 6.5)]), 0.0, [0.6417432743339354 0.31431808695131835; 0.0 

In [15]:
predictions_SVM, decision_values = svmpredict(model_SVM, X[testids,:]')
println("Prediction: $predictions_SVM \nValues $decision_values ")

Prediction: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3] 
Values [1.1022483129200726 1.1748795847202003 1.163429153133285 1.1111039844270567 0.8989464301073786 1.187327374866412 1.1732177042051546 1.129473890413752 1.0529954542072324 1.124325993877869 1.194853772606267 1.0563121406488807 0.7085759668272678 1.1114719962780617 1.003016648285098 1.1431286621281285 -1.0020921590812961 -1.1228120786129168 -1.104863190957336 -1.1886987661621118 -1.1179762038118535 -1.2394969398589322 -1.198731512414002 -1.1308361336876973 -1.1566441702472463 -1.0938576990692632 -1.2373328937734478 -0.9946217076873354 -1.2648974445154924 -0.9318099709961771 -0.4928030433280016 -0.7957468686813252 -0.8470596912078557 -0.6776603286217651 -0.7189846377675443 -0.7570845115709965 -0.7138204475459949 -0.3986497410008372 -0.8063709239676542 -0.8260535690745795 -0.6331649873197341 -0.8241498647568806 -0.9668672893731761 -0.7381

In [16]:
findaccuracy(predictions_SVM,y[testids])

0.9777777777777777

In [20]:
SVM = findaccuracy(predictions_SVM,y[testids])
RandomForest = findaccuracy(pred_RF, y[testids])
DTree = findaccuracy(pred_DT,y[testids])

0.9555555555555556

In [22]:
println(
    "Accuracy:
    SVM = $SVM
    RandomForest = $RandomForest
    DecisionTree = $DTree"
)

Accuracy:
    SVM = 0.9777777777777777
    RandomForest = 0.9777777777777777
    DecisionTree = 0.9555555555555556
